In [11]:
import os
import numpy as np
import pandas as pd
import psycopg2 

In [12]:
os.getcwd()

'C:\\Users\\Arpit Akar'

In [12]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is CEBB-78D3

 Directory of C:\Users\Arpit Akar

11-04-2023  23:39    <DIR>          .
11-04-2023  23:39    <DIR>          ..
10-03-2023  17:30    <DIR>          .astropy
10-03-2023  15:01    <DIR>          .conda
26-01-2023  09:14    <DIR>          .continuum
11-04-2023  22:30    <DIR>          .ipynb_checkpoints
26-01-2023  09:15    <DIR>          .ipython
27-01-2023  21:45    <DIR>          .jupyter
26-01-2023  09:15    <DIR>          .matplotlib
15-03-2023  17:57                56 .scala_history
29-03-2023  22:58    <DIR>          .spyder-py3
21-01-2023  07:39    <DIR>          3D Objects
11-04-2023  22:56             7,401 API_programs.ipynb
29-03-2023  22:51               453 client.py
11-04-2023  18:46            26,511 client_compueter.ipynb
21-01-2023  07:39    <DIR>          Contacts
12-03-2023  00:07            90,448 COREY_vro.ipynb
30-11-2020  00:34             3,243 csv_import_functions.py
30-11-2020  00:34           

In [13]:
def csv_files():
    csv_files=[]
    for file in os.listdir(os.getcwd()):
        if file.endswith('.csv'):
            csv_files.append(file)
    return csv_files

In [14]:
def configure_dataset_directory(csv_files,dataset_dir):
    #make directory
    try:
        mkdir ='mkdir {0}'.format(dataset_dir)
        os.system(mkdir)
    except:
        pass
    # move files into directory
    for csv in csv_files:
        mv_file="mv '{0}' {1}".format(csv,dataset_dir)
        os.system(mv_file)
        print(mv_file)
    return

In [16]:
dataset_dir= 'datasets'
csvies= csv_files()
configure_dataset_directory(csvies,dataset_dir)
df=create_df(dataset_dir,csvies)

In [15]:
def create_df(dataset_dir,csv_files):
    #path to csv files
    data_path = os.getcwd()+'/'+dataset_dir+'/'
    df={}
    # loop through the files and create the dataframe
    for file in csv_files:
        try:
            df[file]=pd.read_csv(data_path+file)
        except UnicodeDecodeError:
            df[file]=pd.read_csv(data_path+file,encoding="ISO-8859-1")
    return df
        

In [17]:
df

{}

In [19]:
def clean_tbl_name(filename):
    clean_tbl_name=filename.lower().replace(' ','_').replace('$','').replace(r'/','')\
                        .replace('%','')
    tbl_name = '{0}'.format(clean_tbl_name.split('.')[0])
    # {0} inputing from format split on dot csv and take left part byindexing it to zero
    return  tbl_name

In [22]:
def clean_colname(data_frame):    
    data_frame.columns=[x.lower().replace(' ','_').replace('$','').replace('/','')\
                        .replace('%','').replace(r"(","") for x in data_frame.columns ]

    replacements={
        'object':'varchar',
        'float64':'float',
        'int64':'int',
        'datetime64':'timestamp',
        'timedelta4[ns]':'varchar'
    }
        #table schema
    col_str=", ".join("{} {}".format(n,d) for (n,d) in zip(data_frame.columns,data_frame.dtypes.replace(replacements)))
    return col_str,data_frame.columns

In [24]:
#open a databse conncetion
host='http://localhost:8888/notebooks/client_compueter.ipynb'
dbname= 'natedb'
user= 'nate'
password='XXXX'
for k in csv_files:
    #call dataframe
    data_frame=df[k]
    # clean table name
    tbl_name=clean_tbl_name(k)
    # clean column names
    col_str,data_frame.columns=clean_columns(data_frame)
    upload_to_db(host,dbname,user,password,tbl_name,col_str,file=k,data_frame,data_frame=data_frame.columns)
    

TypeError: 'function' object is not iterable

In [ ]:
def upload_to_db(host,dbname,user,password,tbl_name,col_str,file,data_frame,data_frame=data_frame.columns):
    conn_string="host=%s \   #adress to the database
                 dbname = %s \ #db name
                 user=%s password=%s "%(host,dbname,user,password) #db_password
    conn= psycopg2.connect(conn_string)
    cursor = conn.cursor()
    print('opened databse successfully') 
    #create table
     cursor.execute("create table %s (%s)" %(tbl_name,col_str))
    conn.commit()
    conn.close()
    #insert value to table


    #save df to csv
    df.to_csv('customer_contracts.csv',header=data_frame,index=False,encoding=('utf-8'))
    # headers are df.columns and we don't want indexes from pandas df to csv file encoding 
    #ENCODING during import from exvel there ma be strange symbols that we say that csv file we have have encoding utf-8
    #open the csv file
    my_file=open[file]
    print('file opened in memory')
    #upload to db
    SQL_STATEMENT= """
    copy= %s from STDIN WITH 
            CSV
            HEADER
            DELIMITER AS ','
    """
    cursor.copy_expert(sql=SQL_STATEMENT,% tbl_name,file=my_file)
    print('file copied to db')
    cursor.execute('grant select on table %s to public' % tbl_name)
    conn.commit()

    conn.close()
    print('table {0} imported to sb completed'.format(tbl_name))